In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)
import os
os.chdir('./drive/My Drive/TESI/ST-ResNet')

Mounted at /content/drive/


In [2]:
from __future__ import print_function
import os
import sys
import pickle
import time
import numpy as np
import h5py

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

from deepst.models.STResNet import stresnet
import deepst.metrics as metrics
from deepst.datasets import TaxiBJ

In [3]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.4.3
2.3.0
Num GPUs Available:  1


In [4]:
np.random.seed(1337)  # for reproducibility

# parameters
DATAPATH = '../data'  # data path, you may set your own data path with the global envirmental variable DATAPATH
CACHEDATA = True  # cache data or NOT
path_cache = os.path.join(DATAPATH, 'CACHE', 'ST-ResNet')  # cache path
nb_epoch = 100  # number of epoch at training stage
# nb_epoch_cont = 100  # number of epoch at training (cont) stage
batch_size = 32  # batch size
T = 48  # number of time intervals in one day
lr = 0.0002  # learning rate
len_closeness = 3  # length of closeness dependent sequence
len_period = 1  # length of peroid dependent sequence
len_trend = 1  # length of trend dependent sequence
nb_residual_unit = 12 # paper says 12 for taxiBJ

nb_flow = 2  # there are two types of flows: inflow and outflow
# divide data into two subsets: Train & Test, of which the test set is the
# last 4 weeks
days_test = 7 * 4
len_test = T * days_test
map_height, map_width = 32, 32  # grid size
path_result = 'RET'
path_model = 'MODEL'

In [ ]:
if os.path.isdir(path_result) is False:
    os.mkdir(path_result)
if os.path.isdir(path_model) is False:
    os.mkdir(path_model)
if CACHEDATA and os.path.isdir(path_cache) is False:
    os.mkdir(path_cache)

In [5]:
def build_model(external_dim):
    c_conf = (len_closeness, nb_flow, map_height,
              map_width) if len_closeness > 0 else None
    p_conf = (len_period, nb_flow, map_height,
              map_width) if len_period > 0 else None
    t_conf = (len_trend, nb_flow, map_height,
              map_width) if len_trend > 0 else None
    model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf,
                     external_dim=external_dim, nb_residual_unit=nb_residual_unit)
    adam = Adam(lr=lr)
    model.compile(loss='mse', optimizer=adam, metrics=[metrics.rmse])
    model.summary()
    # from keras.utils.visualize_util import plot
    # plot(model, to_file='model.png', show_shapes=True)
    return model

In [6]:
def read_cache(fname):
    mmn = pickle.load(open('preprocessing_taxibj.pkl', 'rb'))

    f = h5py.File(fname, 'r')
    num = int(f['num'].value)
    X_train, Y_train, X_test, Y_test = [], [], [], []
    for i in range(num):
        X_train.append(f['X_train_%i' % i].value)
        X_test.append(f['X_test_%i' % i].value)
    Y_train = f['Y_train'].value
    Y_test = f['Y_test'].value
    external_dim = f['external_dim'].value
    timestamp_train = f['T_train'].value
    timestamp_test = f['T_test'].value
    f.close()

    return X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test

In [7]:
def cache(fname, X_train, Y_train, X_test, Y_test, external_dim, timestamp_train, timestamp_test):
    h5 = h5py.File(fname, 'w')
    h5.create_dataset('num', data=len(X_train))

    for i, data in enumerate(X_train):
        h5.create_dataset('X_train_%i' % i, data=data)
    # for i, data in enumerate(Y_train):
    for i, data in enumerate(X_test):
        h5.create_dataset('X_test_%i' % i, data=data)
    h5.create_dataset('Y_train', data=Y_train)
    h5.create_dataset('Y_test', data=Y_test)
    external_dim = -1 if external_dim is None else int(external_dim)
    h5.create_dataset('external_dim', data=external_dim)
    h5.create_dataset('T_train', data=timestamp_train)
    h5.create_dataset('T_test', data=timestamp_test)
    h5.close()

In [8]:
print("loading data...")
ts = time.time()
fname = os.path.join(path_cache, 'TaxiBJ_C{}_P{}_T{}.h5'.format(
    len_closeness, len_period, len_trend))
if os.path.exists(fname) and CACHEDATA:
    X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = read_cache(
        fname)
    print("load %s successfully" % fname)
else:
    X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = TaxiBJ.load_data(
        T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
        preprocess_name='preprocessing_taxibj.pkl', meta_data=True, meteorol_data=False, holiday_data=False, datapath=DATAPATH)
    if CACHEDATA:
        cache(fname, X_train, Y_train, X_test, Y_test,
              external_dim, timestamp_train, timestamp_test)

print("\n days (test): ", [v[:8] for v in timestamp_test[0::T]])
print("\nelapsed time (loading data): %.3f seconds\n" % (time.time() - ts))

print('=' * 10)


loading data...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  


load ../data/CACHE/ST-ResNet/TaxiBJ_C3_P1_T1.h5 successfully

 days (test):  [b'20160310', b'20160311', b'20160312', b'20160313', b'20160314', b'20160315', b'20160316', b'20160317', b'20160318', b'20160319', b'20160320', b'20160321', b'20160322', b'20160325', b'20160326', b'20160327', b'20160328', b'20160329', b'20160331', b'20160401', b'20160402', b'20160403', b'20160404', b'20160405', b'20160406', b'20160407', b'20160408', b'20160409']

elapsed time (loading data): 9.498 seconds



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: H5pyDepr

In [10]:
#from deepst.datasets import stat; stat('BJ16_M32x32_T30_InOut.h5')
f = h5py.File('../data/TaxiBJ/BJ15_M32x32_T30_InOut.h5', 'r')
print(len(f['date']))
# print(f['data'][1][0][0])

print(len(X_train))
print(len(X_train[0]))
print(len(X_train[0][0]))
xtrain_np = np.array(X_train[0], dtype=np.float32)
print(xtrain_np.shape)
print(len(Y_train))
ytrain_np = np.array(Y_train, dtype=np.float32)
print(ytrain_np.shape)
print(len(timestamp_train))
print(len(timestamp_test))
print(len(X_test))
X_train[0].mean()

5596
4
13728
6
(13728, 6, 32, 32)
13728
(13728, 2, 32, 32)
13728
1344
4


-0.8367472361048437

In [11]:
print("compiling model...")
print(
    "**at the first time, it takes a few minites to compile if you use [Theano] as the backend**")
ts = time.time()
model = build_model(external_dim)
hyperparams_name = 'TaxiBJ.c{}.p{}.t{}.resunit{}.lr{}'.format(
    len_closeness, len_period, len_trend, nb_residual_unit, lr)
fname_param = os.path.join('MODEL', '{}.best.h5'.format(hyperparams_name))

early_stopping = EarlyStopping(monitor='val_rmse', patience=2, mode='min')
model_checkpoint = ModelCheckpoint(
    fname_param, monitor='val_rmse', verbose=0, save_best_only=True, mode='min')

print("\nelapsed time (compiling model): %.3f seconds\n" %
      (time.time() - ts))

print('=' * 10)

compiling model...
**at the first time, it takes a few minites to compile if you use [Theano] as the backend**
(None, 2, 32, 32)
(None, 2, 32, 32)
(None, 2, 32, 32)
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 6, 32, 32)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2, 32, 32)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2, 32, 32)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)      

In [12]:
print("training model...")
ts = time.time()
history = model.fit(X_train, Y_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[model_checkpoint],
                    # callbacks=[early_stopping, model_checkpoint],
                    verbose=1)
model.save_weights(os.path.join(
    'MODEL', '{}.h5'.format(hyperparams_name)), overwrite=True)
pickle.dump((history.history), open(os.path.join(
    path_result, '{}.history.pkl'.format(hyperparams_name)), 'wb'))
print("\nelapsed time (training): %.3f seconds\n" % (time.time() - ts))


training model...
Epoch 1/100
387/387 [==============================] - 57s 147ms/step - loss: 0.0247 - rmse: 0.1292 - val_loss: 0.0044 - val_rmse: 0.0650
Epoch 2/100
387/387 [==============================] - 57s 148ms/step - loss: 0.0030 - rmse: 0.0540 - val_loss: 0.0026 - val_rmse: 0.0495
Epoch 3/100
387/387 [==============================] - 58s 149ms/step - loss: 0.0022 - rmse: 0.0463 - val_loss: 0.0022 - val_rmse: 0.0464
Epoch 4/100
387/387 [==============================] - 58s 150ms/step - loss: 0.0018 - rmse: 0.0416 - val_loss: 0.0018 - val_rmse: 0.0415
Epoch 5/100
387/387 [==============================] - 57s 147ms/step - loss: 0.0016 - rmse: 0.0395 - val_loss: 0.0022 - val_rmse: 0.0463
Epoch 6/100
387/387 [==============================] - 57s 147ms/step - loss: 0.0015 - rmse: 0.0380 - val_loss: 0.0019 - val_rmse: 0.0435
Epoch 7/100
387/387 [==============================] - 58s 150ms/step - loss: 0.0013 - rmse: 0.0361 - val_loss: 0.0014 - val_rmse: 0.0369
Epoch 8/100
387/

In [14]:
print('evaluating using the model that has the best loss on the valid set')

model.load_weights(fname_param)
score = model.evaluate(
    X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
        (score[0], score[1], score[1] * (1292 - mmn._min) / 2.))

evaluating using the model that has the best loss on the valid set
Test score: 0.000695 rmse (norm): 0.026367 rmse (real): 17.033243
